# world bank country data

This is a file to download relevant data from worldbank. This should be run using docker image from `dkwok/python-workspace` for preinstalled packages (depends on `kaggle/python` image)

## setup

Import relevant packages and some helper functions

In [175]:
import requests
import pprint
import pandas as pd
import wbdata
import matplotlib.pyplot as plt
import numpy as np

In [44]:
pp = pprint.PrettyPrinter(indent=4)

## query 

use the `wbdata` python package to query the worldbank API

In [78]:
countries = ["AR","BR","CO","CL","MX","PE"]

In [10]:
indicators = {
    'NY.GNP.PCAP.CD':'gni_per_capita',
    'NY.GDP.MKTP.CD':'gdp',
    'SP.POP.TOTL':'pop',
    'SP.URB.TOTL':'pop_urban',
    'SP.POP.0014.TO.ZS':'pop_0-14',
    'SP.POP.1564.TO.ZS':'pop_15-64',
    'SP.POP.65UP.TO.ZS':'pop_65+',
    'SL.TLF.TOTL.IN':'labor_force',
    'SH.DYN.MORT':'mortality_rate',
    'IT.NET.USER.P2': 'internet_user_per_100_people',
    'IT.NET.BBND': 'fixed_broadband_subs',
    'IT.CEL.SETS': 'cellular_mobile_subs'
}

In [11]:
df = wbdata.get_dataframe(
            indicators,
            country="all",
            convert_date=True,
            keep_levels=True)

In [12]:
dfu = df.reset_index()

In [105]:
print(dfu.tail())

        country       date  cellular_mobile_subs  fixed_broadband_subs  \
14779  Zimbabwe 1964-01-01                   NaN                   NaN   
14780  Zimbabwe 1963-01-01                   NaN                   NaN   
14781  Zimbabwe 1962-01-01                   NaN                   NaN   
14782  Zimbabwe 1961-01-01                   NaN                   NaN   
14783  Zimbabwe 1960-01-01                   0.0                   NaN   

                gdp  gni_per_capita  internet_user_per_100_people  \
14779  1.217138e+09           260.0                           NaN   
14780  1.159512e+09           280.0                           NaN   
14781  1.117602e+09           270.0                           NaN   
14782  1.096647e+09             NaN                           NaN   
14783  1.052990e+09             NaN                           NaN   

       labor_force  mortality_rate        pop   pop_0-14  pop_15-64   pop_65+  \
14779          NaN           132.8  4279561.0  47.465383  4

## prepare the data

For now, because the WB data is not completely reliable, let's prepare the data such that we have the ISO2 code for countries, and also take the most recent data point available for each country

### country iso2 code mapping 

In [110]:
countries = wbdata.get_country(display=False)

In [111]:
d = dict((x['name'].strip(), x['iso2Code']) for x in countries)

In [112]:
pp.pprint(d)

{   'Afghanistan': 'AF',
    'Africa': 'A9',
    'Albania': 'AL',
    'Algeria': 'DZ',
    'American Samoa': 'AS',
    'Andean Region': 'L5',
    'Andorra': 'AD',
    'Angola': 'AO',
    'Antigua and Barbuda': 'AG',
    'Arab World': '1A',
    'Argentina': 'AR',
    'Armenia': 'AM',
    'Aruba': 'AW',
    'Australia': 'AU',
    'Austria': 'AT',
    'Azerbaijan': 'AZ',
    'Bahamas, The': 'BS',
    'Bahrain': 'BH',
    'Bangladesh': 'BD',
    'Barbados': 'BB',
    'Belarus': 'BY',
    'Belgium': 'BE',
    'Belize': 'BZ',
    'Benin': 'BJ',
    'Bermuda': 'BM',
    'Bhutan': 'BT',
    'Bolivia': 'BO',
    'Bosnia and Herzegovina': 'BA',
    'Botswana': 'BW',
    'Brazil': 'BR',
    'British Virgin Islands': 'VG',
    'Brunei Darussalam': 'BN',
    'Bulgaria': 'BG',
    'Burkina Faso': 'BF',
    'Burundi': 'BI',
    'Cabo Verde': 'CV',
    'Cambodia': 'KH',
    'Cameroon': 'CM',
    'Canada': 'CA',
    'Caribbean small states': 'S3',
    'Cayman Islands': 'KY',
    'Central African Republ

In [113]:
dfu2 = dfu.assign(iso_code= dfu.country.map(lambda x: d[x]))

### sort columns

In [126]:
cols = dfu2.columns.tolist()

In [127]:
cols = cols[-1:] + cols[:-1]

In [129]:
dfu3 = dfu4[cols]

### fill NaNs with most recently available data

In [144]:
dfu4 = dfu3.fillna(method='bfill')

### take most recent value 

In [145]:
max_date = max(dfu4.date.unique())

In [146]:
max_date

numpy.datetime64('2015-01-01T00:00:00.000000000')

In [150]:
dfu5 = dfu4[dfu4.date==max_date].reset_index()

In [165]:
print(dfu5.shape)
print(dfu5.head())

(264, 16)
   index iso_code                         country       date  \
0      0       1A                      Arab World 2015-01-01   
1     56       S3          Caribbean small states 2015-01-01   
2    112       B8  Central Europe and the Baltics 2015-01-01   
3    168       V2      Early-demographic dividend 2015-01-01   
4    224       Z4             East Asia & Pacific 2015-01-01   

   cellular_mobile_subs  fixed_broadband_subs           gdp  gni_per_capita  \
0          4.104229e+08            17599667.0  2.530102e+12     7060.619946   
1          8.033917e+06              744280.0  7.087662e+10     9476.994987   
2          1.373719e+08            22838613.0  1.273221e+12    13147.787643   
3          2.658151e+09           109788426.0  9.844617e+12     3324.105287   
4          2.371814e+09           317815911.0  2.128119e+13     9602.441917   

   internet_user_per_100_people   labor_force  mortality_rate           pop  \
0                     34.517375  1.299310e+08      

###  variable cleaning

In [178]:
def fx(x,y): return int(x*y/100)

In [188]:
dfu6 = dfu5.assign(cellular_mobile_subs = dfu5.cellular_mobile_subs.map(lambda x: int(x)),
                   fixed_broadband_subs = dfu5.fixed_broadband_subs.map(lambda x: int(x)),
                   gdp = dfu5.gdp.map(lambda x: int(x)),
                   gni = np.vectorize(fx)(dfu5.gni_per_capita, dfu5['pop']),
                   labor_force = dfu5.labor_force.map(lambda x: int(x)),
                   pop = dfu5['pop'].map(lambda x: int(x)),
                   pop_urban = dfu5.pop_urban.map(lambda x: int(x)),
                   pop_0_14 = np.vectorize(fx)(dfu5['pop_0-14'], dfu5['pop']),
                   pop_15_64 = np.vectorize(fx)(dfu5['pop_15-64'], dfu5['pop']),
                   pop_65_plus = np.vectorize(fx)(dfu5['pop_65+'], dfu5['pop']),
                   internet_users = np.vectorize(fx)(dfu5.internet_user_per_100_people, dfu5['pop']),
                      )

In [189]:
print(dfu6.head())

   index iso_code                         country       date  \
0      0       1A                      Arab World 2015-01-01   
1     56       S3          Caribbean small states 2015-01-01   
2    112       B8  Central Europe and the Baltics 2015-01-01   
3    168       V2      Early-demographic dividend 2015-01-01   
4    224       Z4             East Asia & Pacific 2015-01-01   

   cellular_mobile_subs  fixed_broadband_subs             gdp  gni_per_capita  \
0             410422908              17599667   2530101503617     7060.619946   
1               8033917                744280     70876623388     9476.994987   
2             137371883              22838613   1273220514555    13147.787643   
3            2658150845             109788426   9844617142362     3324.105287   
4            2371814450             317815911  21281190210607     9602.441917   

   internet_user_per_100_people  labor_force     ...              pop  \
0                     34.517375    129930989     ...   

## Export relevant columns

In [195]:
dfu7 = dfu6.drop(['internet_user_per_100_people', 'mortality_rate', 'pop_0-14',
                  'pop_15-64', 'pop_65+', 'gni_per_capita', 'index', 'date'],
                 axis=1)

In [196]:
print(dfu7.head())

  iso_code                         country  cellular_mobile_subs  \
0       1A                      Arab World             410422908   
1       S3          Caribbean small states               8033917   
2       B8  Central Europe and the Baltics             137371883   
3       V2      Early-demographic dividend            2658150845   
4       Z4             East Asia & Pacific            2371814450   

   fixed_broadband_subs             gdp  labor_force         pop   pop_urban  \
0              17599667   2530101503617    129930989   392022276   226673886   
1                744280     70876623388      3228506     7048966     2972747   
2              22838613   1273220514555     49712207   103318638    64424905   
3             109788426   9844617142362   1269209398  3122703317  1378307567   
4             317815911  21281190210607   1259470455  2279186469  1277399664   

            gni  internet_users   pop_0_14   pop_15_64  pop_65_plus  
0   27679203012       135315798  1302650

In [197]:
dfu7.to_csv('wb_data.csv',index=False)